# Feature names

In [17]:
import pandas as pd

In [18]:
test = pd.read_pickle('test_1_agg_imputed.bin')
train = pd.read_pickle('train_1_agg_imputed.bin')

In [20]:
map = pd.read_csv('column_mapping.csv')

In [ ]:
map

In [22]:
dic = map.set_index('Masked Column')['Original Column'].to_dict()

# Product wise stability test

In [ ]:
! uv pip install xgboost scikit-learn lightgbm catboost

In [4]:
import pickle
import pandas as pd

In [5]:
imputed_data = pd.read_pickle('test_1_agg_imputed.bin')
X  = imputed_data.drop(columns=['id', 'loan_id'])
labels = imputed_data['loan_id']

In [ ]:
with open("xgb_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("lgbm_model.pkl", "rb") as f:
    model2 = pickle.load(f)

with open("catboost_model.pkl", "rb") as f:
    model3 = pickle.load(f)

with open("final_calibrated_model.pkl", "rb") as f:
    metamodel = pickle.load(f)

with open("X_test.pkl", "rb") as f:
    X = pickle.load(f)

from sklearn.metrics import roc_auc_score

# Step 1: Predict the probabilities
y_pred = model.predict(X)
y_pred2 = model2.predict(X)
y_pred3 = model3.predict(X)

pred_df = pd.DataFrame({
    'XGB_Preds': y_pred,
    'LGBM_Preds': y_pred2,
    'CatBoost_Preds': y_pred3
})

# Step 2: Meta-model prediction
final_preds = metamodel.predict_proba(pred_df)[:, 1]

In [ ]:
final_preds

In [ ]:
labels

In [9]:
results = pd.DataFrame({
    'Predictions': final_preds,
    'ID': labels
})



In [ ]:
results

In [11]:
classes = pd.read_csv('test_labels.csv')

In [ ]:
classes

In [13]:
final = pd.merge(imputed_data, results, left_on='loan_id', right_on='ID')

In [16]:
labels = pd.read_csv("test_labels.csv")
final = pd.merge(final, labels, left_on='loan_id', right_on='loan_id')

In [ ]:
final

In [18]:
final['label'] = final['label'].astype(int)

In [ ]:
from sklearn.metrics import roc_auc_score
from scipy.stats import ks_2samp

roc_auc = roc_auc_score(final['label'], final['Predictions'])
ks_stat, ks_p = ks_2samp(final.loc[final['label'] == 0, 'Predictions'], final.loc[final['label'] == 1, 'Predictions'])

print(f"ROC AUC Score: {roc_auc}")
print(f"KS Statistic: {ks_stat}, p-value: {ks_p}")


# Product wise breakdown

In [ ]:
final

In [22]:
products = final['prod'].unique()

In [23]:
afinal = final.copy()

In [ ]:
for pr in products:
    final = afinal[afinal['prod'] == pr]
    roc_auc = roc_auc_score(final['label'], final['Predictions'])
    ks_stat, ks_p = ks_2samp(final.loc[final['label'] == 0, 'Predictions'], final.loc[final['label'] == 1, 'Predictions'])

    print(f"Product",pr)

    print(f"ROC AUC Score: {roc_auc}")
    print(f"KS Statistic: {ks_stat}, p-value: {ks_p}")
    print("-----\n")
